# Question 1

1. The cost to the MTA of the "primary" carriers is significantly higher than that of the "broker" or "ehail" trips. Because of the individual needs of each customer though, not all trips can be rendered on
"broker" service. Leadership is interested in making the Paratransit service as cost effective as possible,
and as part of this, is interested in understanding the proportion of trips completed on "primary" vs
"broker". Provide a table showing: the total number of successful trips, the average trips per day, and
the % of trips by mode (i.e., provider type) for weekday vs. weekend. 

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_csv("trips.csv")

In [3]:
df.head()

,TripId,Clientid,Status,ProviderType,Outcome,Tripdate,RequestTime,PromiseTime,ProviderId,APtime,APtime1,ADtime,Anchor,PickCounty,PickZip,DropCounty,DropZip,TripMiles
0,1,18958,CX,E-Hail,Early Cancel,2022-04-01,06:30,05:35,NaN,00:00,00:00,00:00,P,Kings County,11228,Kings County,11219,0.00
1,2,33563,S,E-Hail,Authorized,2022-04-01,23:00,22:03,ISA,00:00,00:00,00:00,A,Richmond County,10308,Richmond County,10307,7.63
2,9,51426,CX,E-Hail,Early Cancel,2022-04-01,15:30,14:48,NaN,00:00,00:00,00:00,P,New York County,10022,Richmond County,10301,0.00
3,20,37638,CX,E-Hail,Early Cancel,2022-04-01,14:30,14:30,NaN,00:00,00:00,00:00,P,Kings,11209,Richmond,10314,0.00
4,21,36285,S,Primary,Completed,2022-04-01,09:30,09:30,MAG,09:38,00:00,09:55,P,Kings,11203,Kings,11226,0.90


In [4]:
df.describe()

,TripId,Clientid,TripMiles
count,1.164013e+06,1.164013e+06,1.164013e+06
mean,5.820057e+05,2.911862e+04,3.362872e+00
std,3.360214e+05,1.716323e+04,5.427824e+00
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.910030e+05,1.423400e+04,0.000000e+00
50%,5.820060e+05,2.958800e+04,0.000000e+00
75%,8.730080e+05,4.385000e+04,5.130000e+00
max,1.164011e+06,5.931200e+04,4.649000e+01


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1164013 entries, 0 to 1164012
Data columns (total 18 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   TripId        1164013 non-null  int64  
 1   Clientid      1164013 non-null  int64  
 2   Status        1164013 non-null  object 
 3   ProviderType  1164013 non-null  object 
 4   Outcome       1164013 non-null  object 
 5   Tripdate      1164013 non-null  object 
 6   RequestTime   1164013 non-null  object 
 7   PromiseTime   1164013 non-null  object 
 8   ProviderId    543476 non-null   object 
 9   APtime        1164012 non-null  object 
 10  APtime1       1164013 non-null  object 
 11  ADtime        1164012 non-null  object 
 12  Anchor        1164013 non-null  object 
 13  PickCounty    1138339 non-null  object 
 14  PickZip       1138336 non-null  object 
 15  DropCounty    1138338 non-null  object 
 16  DropZip       1138336 non-null  object 
 17  TripMiles     1164013 non-n

## Total Number of Successful Trips - Primary, Broker, E-Hail

In [6]:
# Changing datatype to datetime
df["Tripdate"] = pd.to_datetime(df["Tripdate"])

In [7]:
# Creating column 
df["day_of_week"] = df["Tripdate"].dt.day_name()

In [8]:
df["ProviderType"].unique()

array(['E-Hail', 'Primary', 'Broker', 'Deleted', 'Test Record',
       'Driver break record', 'Fixed Route-Exclude'], dtype=object)

In [9]:
# Removing all values except Primary, Broker, and E-Hail
df = df.loc[~(df['ProviderType'].isin(['Driver break record', 'Test Record', 'Fixed Route-Exclude', 'Deleted']))]

In [10]:
# Designating each successful trip based on whether the trip happened on a Weekday or Weekend
def dofweek(df):
    df.loc[(df['day_of_week'] == "Saturday"), 'Weekday/Weekend'] = "Weekend"
    df.loc[(df['day_of_week'] == "Sunday"), 'Weekday/Weekend'] = "Weekend"
    df['Weekday/Weekend'].fillna("Weekday", inplace=True)

In [11]:
dofweek(df)

In [12]:
df

,TripId,Clientid,Status,ProviderType,Outcome,Tripdate,RequestTime,PromiseTime,ProviderId,APtime,APtime1,ADtime,Anchor,PickCounty,PickZip,DropCounty,DropZip,TripMiles,day_of_week,Weekday/Weekend
0,1,18958,CX,E-Hail,Early Cancel,2022-04-01,06:30,05:35,NaN,00:00,00:00,00:00,P,Kings County,11228,Kings County,11219,0.00,Friday,Weekday
1,2,33563,S,E-Hail,Authorized,2022-04-01,23:00,22:03,ISA,00:00,00:00,00:00,A,Richmond County,10308,Richmond County,10307,7.63,Friday,Weekday
2,9,51426,CX,E-Hail,Early Cancel,2022-04-01,15:30,14:48,NaN,00:00,00:00,00:00,P,New York County,10022,Richmond County,10301,0.00,Friday,Weekday
3,20,37638,CX,E-Hail,Early Cancel,2022-04-01,14:30,14:30,NaN,00:00,00:00,00:00,P,Kings,11209,Richmond,10314,0.00,Friday,Weekday
4,21,36285,S,Primary,Completed,2022-04-01,09:30,09:30,MAG,09:38,00:00,09:55,P,Kings,11203,Kings,11226,0.90,Friday,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164007,68975,56090,TAXI,E-Hail,Authorized,2022-04-01,16:15,00:00,NaN,00:00,00:00,00:00,P,Kings,11232,Kings,11204,0.00,Friday,Weekday
1164008,341231,52945,TAXI,E-Hail,Authorized,2022-04-08,15:20,00:00,NaN,00:00,00:00,00:00,P,Queens,11361,Queens,11358,0.00,Friday,Weekday
1164009,564342,613,S,Primary,Completed,2022-04-14,09:47,09:47,MVP,09:46,09:33,10:16,P,Bronx,10475,Westchester,10701,7.65,Thursday,Weekday
1164011,324756,40067,S,Primary,Completed,2022-04-08,12:15,12:15,GVC,12:25,11:54,12:47,P,New York,10065,New York,10003,3.39,Friday,Weekday


In [13]:
# Subsetting data for only successful trips
df2 = df[df["Status"] == "S"]

In [14]:
df2

,TripId,Clientid,Status,ProviderType,Outcome,Tripdate,RequestTime,PromiseTime,ProviderId,APtime,APtime1,ADtime,Anchor,PickCounty,PickZip,DropCounty,DropZip,TripMiles,day_of_week,Weekday/Weekend
1,2,33563,S,E-Hail,Authorized,2022-04-01,23:00,22:03,ISA,00:00,00:00,00:00,A,Richmond County,10308,Richmond County,10307,7.630000,Friday,Weekday
4,21,36285,S,Primary,Completed,2022-04-01,09:30,09:30,MAG,09:38,00:00,09:55,P,Kings,11203,Kings,11226,0.900000,Friday,Weekday
5,38,38611,S,Primary,Completed,2022-04-01,11:15,11:15,MVT,11:20,11:15,12:46,P,Kings,11203,New York,10004,8.310000,Friday,Weekday
6,39,38611,S,Primary,Completed,2022-04-01,21:10,21:10,MVT,21:42,21:40,23:03,P,New York,10004,Kings,11203,8.310000,Friday,Weekday
7,53,35870,S,Broker,Authorized,2022-04-01,12:30,12:30,VBH,12:44,12:14,13:57,P,New York,10004,Queens,11426,16.370001,Friday,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164001,196088,32341,S,Broker,Authorized,2022-04-04,20:10,20:10,VBG,00:00,20:30,20:52,P,New York,10032,Bronx,10461,5.490000,Monday,Weekday
1164002,670483,28744,S,E-Hail,Authorized,2022-04-17,09:59,09:59,LIS,00:00,00:00,00:00,P,Kings,11233,Queens,11419,6.230000,Sunday,Weekend
1164005,155313,34239,S,E-Hail,Authorized,2022-04-04,12:15,12:15,LIS,00:00,00:00,00:00,P,New York,10065,Queens,11434,16.570000,Monday,Weekday
1164009,564342,613,S,Primary,Completed,2022-04-14,09:47,09:47,MVP,09:46,09:33,10:16,P,Bronx,10475,Westchester,10701,7.650000,Thursday,Weekday


In [15]:
new_df2 = df2.groupby(["ProviderType", "Weekday/Weekend"]).count().reset_index()

In [16]:
new_df3 = new_df2[["ProviderType", "Weekday/Weekend", "day_of_week"]]
new_df3

,ProviderType,Weekday/Weekend,day_of_week
0,Broker,Weekday,230605
1,Broker,Weekend,58012
2,E-Hail,Weekday,53009
3,E-Hail,Weekend,9592
4,Primary,Weekday,112506
5,Primary,Weekend,27153


In [17]:
new_df3.rename(columns={"day_of_week": "Total Successful Trips"}, inplace=True)
new_df3

/Users/mohamedhassan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,ProviderType,Weekday/Weekend,Total Successful Trips
0,Broker,Weekday,230605
1,Broker,Weekend,58012
2,E-Hail,Weekday,53009
3,E-Hail,Weekend,9592
4,Primary,Weekday,112506
5,Primary,Weekend,27153


In [18]:
new_df3['Total ProviderType Trips'] = new_df3.groupby('ProviderType')['Total Successful Trips'].transform('sum')

<ipython-input-18-72be5ee74616>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df3['Total ProviderType Trips'] = new_df3.groupby('ProviderType')['Total Successful Trips'].transform('sum')


In [19]:
new_df3['Total Weekday/Weekend Trips'] = new_df3.groupby('Weekday/Weekend')['Total Successful Trips'].transform('sum')

<ipython-input-19-afb6c422f151>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df3['Total Weekday/Weekend Trips'] = new_df3.groupby('Weekday/Weekend')['Total Successful Trips'].transform('sum')


In [20]:
new_df3

,ProviderType,Weekday/Weekend,Total Successful Trips,Total ProviderType Trips,Total Weekday/Weekend Trips
0,Broker,Weekday,230605,288617,396120
1,Broker,Weekend,58012,288617,94757
2,E-Hail,Weekday,53009,62601,396120
3,E-Hail,Weekend,9592,62601,94757
4,Primary,Weekday,112506,139659,396120
5,Primary,Weekend,27153,139659,94757


## Average Trips Per Day

In [21]:
average_trips_per_day = df[["ProviderType", "Weekday/Weekend", "day_of_week"]]
average_trips_per_day

,ProviderType,Weekday/Weekend,day_of_week
0,E-Hail,Weekday,Friday
1,E-Hail,Weekday,Friday
2,E-Hail,Weekday,Friday
3,E-Hail,Weekday,Friday
4,Primary,Weekday,Friday
...,...,...,...
1164007,E-Hail,Weekday,Friday
1164008,E-Hail,Weekday,Friday
1164009,Primary,Weekday,Thursday
1164011,Primary,Weekday,Friday


In [22]:
new_df3['Average Weekday/Weekend Trips'] = new_df3['Total Successful Trips'] /new_df3['Total Weekday/Weekend Trips']

<ipython-input-22-2967a8063369>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df3['Average Weekday/Weekend Trips'] = new_df3['Total Successful Trips'] /new_df3['Total Weekday/Weekend Trips']


In [23]:
new_df3

,ProviderType,Weekday/Weekend,Total Successful Trips,Total ProviderType Trips,Total Weekday/Weekend Trips,Average Weekday/Weekend Trips
0,Broker,Weekday,230605,288617,396120,0.582159
1,Broker,Weekend,58012,288617,94757,0.612219
2,E-Hail,Weekday,53009,62601,396120,0.133821
3,E-Hail,Weekend,9592,62601,94757,0.101227
4,Primary,Weekday,112506,139659,396120,0.284020
5,Primary,Weekend,27153,139659,94757,0.286554


## Percent of Trips by Provider Type

In [24]:
new_df3['% of Trips by ProviderType'] = new_df3['Total Successful Trips'] /new_df3['Total ProviderType Trips']

<ipython-input-24-fbe5e7843b23>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df3['% of Trips by ProviderType'] = new_df3['Total Successful Trips'] /new_df3['Total ProviderType Trips']


In [25]:
new_df3

,ProviderType,Weekday/Weekend,Total Successful Trips,Total ProviderType Trips,Total Weekday/Weekend Trips,Average Weekday/Weekend Trips,% of Trips by ProviderType
0,Broker,Weekday,230605,288617,396120,0.582159,0.799000
1,Broker,Weekend,58012,288617,94757,0.612219,0.201000
2,E-Hail,Weekday,53009,62601,396120,0.133821,0.846776
3,E-Hail,Weekend,9592,62601,94757,0.101227,0.153224
4,Primary,Weekday,112506,139659,396120,0.284020,0.805576
5,Primary,Weekend,27153,139659,94757,0.286554,0.194424


# Question 2

## Primary Trip Pickups by Top 10 Zipcodes

2. Due to the perceived high contract cost of existing "primary" providers, the MTA is looking to start a
new round procurement for additional providers. Current operational practice suggests that the location
of a provider's depot has a significant impact on the amount of time that a vehicle spends with no
customers riding in it, due to the driving time necessary to get from the depot, to the pickup of their first
customer. This "deadhead" time is costly to us, and of no benefit to our customers.
To assist our budget and procurement departments in determining whether or not they should include a
"geographic score" component in the cost proposal review, provide a list of the top 10 ZIP codes where
successful Primary trip pickups occur during the AM rush hour (06:00 through 10:00). Please include a
count/ total of the number of trips as well. Since this question is future looking, please use the time that
best describes when customers were scheduled to be picked up, and not when our vehicles may have
arrived. 

In [26]:
primary_df = df2.loc[~(df2['ProviderType'].isin(['E-Hail', 'Broker']))]
primary_df

,TripId,Clientid,Status,ProviderType,Outcome,Tripdate,RequestTime,PromiseTime,ProviderId,APtime,APtime1,ADtime,Anchor,PickCounty,PickZip,DropCounty,DropZip,TripMiles,day_of_week,Weekday/Weekend
4,21,36285,S,Primary,Completed,2022-04-01,09:30,09:30,MAG,09:38,00:00,09:55,P,Kings,11203,Kings,11226,0.90,Friday,Weekday
5,38,38611,S,Primary,Completed,2022-04-01,11:15,11:15,MVT,11:20,11:15,12:46,P,Kings,11203,New York,10004,8.31,Friday,Weekday
6,39,38611,S,Primary,Completed,2022-04-01,21:10,21:10,MVT,21:42,21:40,23:03,P,New York,10004,Kings,11203,8.31,Friday,Weekday
8,56,4798,S,Primary,Completed,2022-04-01,07:15,07:15,MVT,07:20,07:14,07:32,P,Kings,11221,Kings,11213,1.13,Friday,Weekday
9,70,51875,S,Primary,Completed,2022-04-01,09:45,09:45,ITA,10:01,09:57,11:03,P,Queens,11429,New York,10031,19.93,Friday,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163940,498805,56823,S,Primary,Completed,2022-04-12,15:25,13:15,PTC,13:14,13:14,15:23,P,New York,10035,Bronx,10469,9.52,Tuesday,Weekday
1163945,177165,2751,S,Primary,Completed,2022-04-04,16:30,16:30,MVT,16:47,16:43,17:54,P,Kings,11235,New York,10023,15.21,Monday,Weekday
1163986,196148,11100,S,Primary,Completed,2022-04-04,22:00,22:00,MVT,22:08,21:54,23:00,P,Queens,11040,Kings,11225,19.08,Monday,Weekday
1164009,564342,613,S,Primary,Completed,2022-04-14,09:47,09:47,MVP,09:46,09:33,10:16,P,Bronx,10475,Westchester,10701,7.65,Thursday,Weekday


In [27]:
# Using PromiseTime column to subset data; removing ":"
primary_df["PromiseTime"] = primary_df["PromiseTime"].str.replace(":","")

<ipython-input-27-b400e4714b18>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_df["PromiseTime"] = primary_df["PromiseTime"].str.replace(":","")


In [28]:
primary_df

,TripId,Clientid,Status,ProviderType,Outcome,Tripdate,RequestTime,PromiseTime,ProviderId,APtime,APtime1,ADtime,Anchor,PickCounty,PickZip,DropCounty,DropZip,TripMiles,day_of_week,Weekday/Weekend
4,21,36285,S,Primary,Completed,2022-04-01,09:30,0930,MAG,09:38,00:00,09:55,P,Kings,11203,Kings,11226,0.90,Friday,Weekday
5,38,38611,S,Primary,Completed,2022-04-01,11:15,1115,MVT,11:20,11:15,12:46,P,Kings,11203,New York,10004,8.31,Friday,Weekday
6,39,38611,S,Primary,Completed,2022-04-01,21:10,2110,MVT,21:42,21:40,23:03,P,New York,10004,Kings,11203,8.31,Friday,Weekday
8,56,4798,S,Primary,Completed,2022-04-01,07:15,0715,MVT,07:20,07:14,07:32,P,Kings,11221,Kings,11213,1.13,Friday,Weekday
9,70,51875,S,Primary,Completed,2022-04-01,09:45,0945,ITA,10:01,09:57,11:03,P,Queens,11429,New York,10031,19.93,Friday,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163940,498805,56823,S,Primary,Completed,2022-04-12,15:25,1315,PTC,13:14,13:14,15:23,P,New York,10035,Bronx,10469,9.52,Tuesday,Weekday
1163945,177165,2751,S,Primary,Completed,2022-04-04,16:30,1630,MVT,16:47,16:43,17:54,P,Kings,11235,New York,10023,15.21,Monday,Weekday
1163986,196148,11100,S,Primary,Completed,2022-04-04,22:00,2200,MVT,22:08,21:54,23:00,P,Queens,11040,Kings,11225,19.08,Monday,Weekday
1164009,564342,613,S,Primary,Completed,2022-04-14,09:47,0947,MVP,09:46,09:33,10:16,P,Bronx,10475,Westchester,10701,7.65,Thursday,Weekday


In [29]:
# subsetting data between 6AM and 10AM
new_primary_df = primary_df.loc[primary_df['PromiseTime'].between('0600', '1000')]

In [30]:
new_primary_df

,TripId,Clientid,Status,ProviderType,Outcome,Tripdate,RequestTime,PromiseTime,ProviderId,APtime,APtime1,ADtime,Anchor,PickCounty,PickZip,DropCounty,DropZip,TripMiles,day_of_week,Weekday/Weekend
4,21,36285,S,Primary,Completed,2022-04-01,09:30,0930,MAG,09:38,00:00,09:55,P,Kings,11203,Kings,11226,0.90,Friday,Weekday
8,56,4798,S,Primary,Completed,2022-04-01,07:15,0715,MVT,07:20,07:14,07:32,P,Kings,11221,Kings,11213,1.13,Friday,Weekday
9,70,51875,S,Primary,Completed,2022-04-01,09:45,0945,ITA,10:01,09:57,11:03,P,Queens,11429,New York,10031,19.93,Friday,Weekday
11,88,21318,S,Primary,Completed,2022-04-01,07:30,0730,MVT,07:31,07:30,09:01,P,Kings,11236,Kings,11217,7.19,Friday,Weekday
20,156,36868,S,Primary,Completed,2022-04-01,10:00,1000,PTC,10:05,09:56,10:51,P,New York,10016,Kings,11206,5.28,Friday,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163718,433208,312,S,Primary,Completed,2022-04-11,10:00,1000,PTC,10:00,10:00,10:17,P,Queens,11374,Queens,11421,1.60,Monday,Weekday
1163816,523007,10243,S,Primary,Completed,2022-04-13,08:00,0824,MAG,08:26,08:22,09:27,P,Queens,11374,New York,10001,8.79,Wednesday,Weekday
1163921,150252,16274,S,Primary,Completed,2022-04-04,06:00,0629,PTC,06:40,06:37,08:00,P,Queens,11435,Kings,11238,9.76,Monday,Weekday
1163937,747912,23189,S,Primary,Completed,2022-04-19,09:30,0930,MVT,09:34,09:27,10:11,P,Kings,11234,Kings,11226,3.28,Tuesday,Weekday


In [31]:
new_primary_df.groupby('Weekday/Weekend').count().sort_values(by='PickZip', ascending=False).reset_index()
new_primary_df

,TripId,Clientid,Status,ProviderType,Outcome,Tripdate,RequestTime,PromiseTime,ProviderId,APtime,APtime1,ADtime,Anchor,PickCounty,PickZip,DropCounty,DropZip,TripMiles,day_of_week,Weekday/Weekend
4,21,36285,S,Primary,Completed,2022-04-01,09:30,0930,MAG,09:38,00:00,09:55,P,Kings,11203,Kings,11226,0.90,Friday,Weekday
8,56,4798,S,Primary,Completed,2022-04-01,07:15,0715,MVT,07:20,07:14,07:32,P,Kings,11221,Kings,11213,1.13,Friday,Weekday
9,70,51875,S,Primary,Completed,2022-04-01,09:45,0945,ITA,10:01,09:57,11:03,P,Queens,11429,New York,10031,19.93,Friday,Weekday
11,88,21318,S,Primary,Completed,2022-04-01,07:30,0730,MVT,07:31,07:30,09:01,P,Kings,11236,Kings,11217,7.19,Friday,Weekday
20,156,36868,S,Primary,Completed,2022-04-01,10:00,1000,PTC,10:05,09:56,10:51,P,New York,10016,Kings,11206,5.28,Friday,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163718,433208,312,S,Primary,Completed,2022-04-11,10:00,1000,PTC,10:00,10:00,10:17,P,Queens,11374,Queens,11421,1.60,Monday,Weekday
1163816,523007,10243,S,Primary,Completed,2022-04-13,08:00,0824,MAG,08:26,08:22,09:27,P,Queens,11374,New York,10001,8.79,Wednesday,Weekday
1163921,150252,16274,S,Primary,Completed,2022-04-04,06:00,0629,PTC,06:40,06:37,08:00,P,Queens,11435,Kings,11238,9.76,Monday,Weekday
1163937,747912,23189,S,Primary,Completed,2022-04-19,09:30,0930,MVT,09:34,09:27,10:11,P,Kings,11234,Kings,11226,3.28,Tuesday,Weekday


In [32]:
new_primary_df2 = new_primary_df.groupby('PickZip')['Weekday/Weekend'].count().sort_values(ascending=False).reset_index()
new_primary_df2.head(10)

,PickZip,Weekday/Weekend
0,10314,1220
1,11212,905
2,10304,898
3,11207,859
4,10301,792
5,10306,781
6,11434,732
7,11233,664
8,11691,646
9,11236,614


# Question 3

3. Paratransit customers and advocates for persons with disabilities often criticize the inflexibility of the
Access-A-Ride service compared to the high frequency of our Subways and Buses. For example, a
customer who is reliant on Paratransit for mobility has to know by 5pm the night before exactly what
time they'd like to go from A (home) to B (work), and B (work) back to A (home). Advocates have long
requested "B-leg flexibility". This feature would allow customers who "subscribe" to the same "A-trip"
each day (think: regular commuters), to request a "B-trip" on much shorter notice, say 2-hours prior to
their trip rather than by 5pm the night before. Also, consider that the cost to the MTA scales quite
closely with the distance of the trip.
To assist with cost forecasting of a “flexible B-leg” offering, calculate (for successfully delivered trips) the
following, by 5 digit ZIP code, for trips beginning or ending in ZIPs: 11201, 10469, 11694, 10306
- The mean trip distance, and total number of trips, for trips starting or ending in the respective ZIP
code, that ALSO ended or started within the same borough as that ZIP code (i.e., so called “intraborough” trips)
- The mean trip distance, and total number of trips, for trips starting or ending in the respective ZIP
code, that ended or started or ended in Manhattan (i.e., “inter-borough” trips)
- The overall mean trip distance and total number of trips for trips starting or ending in the
respective ZIP code.
In a few sentences -- comments in the code are OK -- given the results from these 4 ZIP codes, what can
you say about how costly it is to serve Paratransit customers based on their trip geography? 

Hint: there are only 5 boroughs/counties in New York City. Trips beginning or ending outside of those 5
boroughs/counties can be excluded from analysis. 

In [33]:
df2

,TripId,Clientid,Status,ProviderType,Outcome,Tripdate,RequestTime,PromiseTime,ProviderId,APtime,APtime1,ADtime,Anchor,PickCounty,PickZip,DropCounty,DropZip,TripMiles,day_of_week,Weekday/Weekend
1,2,33563,S,E-Hail,Authorized,2022-04-01,23:00,22:03,ISA,00:00,00:00,00:00,A,Richmond County,10308,Richmond County,10307,7.630000,Friday,Weekday
4,21,36285,S,Primary,Completed,2022-04-01,09:30,09:30,MAG,09:38,00:00,09:55,P,Kings,11203,Kings,11226,0.900000,Friday,Weekday
5,38,38611,S,Primary,Completed,2022-04-01,11:15,11:15,MVT,11:20,11:15,12:46,P,Kings,11203,New York,10004,8.310000,Friday,Weekday
6,39,38611,S,Primary,Completed,2022-04-01,21:10,21:10,MVT,21:42,21:40,23:03,P,New York,10004,Kings,11203,8.310000,Friday,Weekday
7,53,35870,S,Broker,Authorized,2022-04-01,12:30,12:30,VBH,12:44,12:14,13:57,P,New York,10004,Queens,11426,16.370001,Friday,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164001,196088,32341,S,Broker,Authorized,2022-04-04,20:10,20:10,VBG,00:00,20:30,20:52,P,New York,10032,Bronx,10461,5.490000,Monday,Weekday
1164002,670483,28744,S,E-Hail,Authorized,2022-04-17,09:59,09:59,LIS,00:00,00:00,00:00,P,Kings,11233,Queens,11419,6.230000,Sunday,Weekend
1164005,155313,34239,S,E-Hail,Authorized,2022-04-04,12:15,12:15,LIS,00:00,00:00,00:00,P,New York,10065,Queens,11434,16.570000,Monday,Weekday
1164009,564342,613,S,Primary,Completed,2022-04-14,09:47,09:47,MVP,09:46,09:33,10:16,P,Bronx,10475,Westchester,10701,7.650000,Thursday,Weekday


In [34]:
df2['PickZip'] = df2['PickZip'].str[:5]
df2['DropZip'] = df2['DropZip'].str[:5]

<ipython-input-34-95750dba3da9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['PickZip'] = df2['PickZip'].str[:5]
<ipython-input-34-95750dba3da9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['DropZip'] = df2['DropZip'].str[:5]


In [35]:
df2 = df2.dropna(subset=['PickZip'])
df2 = df2.dropna(subset=['DropZip'])

In [36]:
df2[['PickZip', 'DropZip']] = df2[['PickZip', 'DropZip']].astype('int64')

In [37]:
df2['PickCounty'].unique()

array(['Richmond County', 'Kings', 'New York', 'Queens', 'Kings County',
       'Richmond', 'Bronx', 'Bronx County', 'Queens County', 'Nassau',
       'Westchester', 'New York County', 'Nassau County',
       'Westchester County', 'J', '--', '3/4TH AVES'], dtype=object)

In [38]:
df2['DropCounty'].unique()

array(['Richmond County', 'Kings', 'New York', 'Queens', 'Kings County',
       'Richmond', 'Bronx', 'Bronx County', 'New York County',
       'Queens County', 'Nassau', 'Westchester', 'Nassau County',
       'Westchester County', '--', 'NEW YORK COUNTY', '3/4TH AVES',
       'KINGS', 'J'], dtype=object)

In [39]:
df2['DropCounty'] = df2['DropCounty'].str.replace('NEW YORK COUNTY', 'New York County')

In [40]:
df2['DropCounty'] = df2['DropCounty'].str.replace('KINGS', 'Kings')

In [41]:
# Removing County from each value
df2['PickCounty'] = df2['PickCounty'].str.replace('County', '')

In [42]:
df2['DropCounty'] = df2['DropCounty'].str.replace('County', '')

In [43]:
# Subsetting data to only include 5 boroughs of New York City for Pickup County
df3 = df2.loc[df2["PickCounty"].isin(['New York', 'Kings', 'Queens', 'Bronx', 'Richmond'])]

In [44]:
# Subsetting data to only include 5 boroughs of New York City for Dropoff County
df4 = df3.loc[df3["DropCounty"].isin(['New York', 'Kings', 'Queens', 'Bronx', 'Richmond'])]

In [45]:
df4

,TripId,Clientid,Status,ProviderType,Outcome,Tripdate,RequestTime,PromiseTime,ProviderId,APtime,APtime1,ADtime,Anchor,PickCounty,PickZip,DropCounty,DropZip,TripMiles,day_of_week,Weekday/Weekend
4,21,36285,S,Primary,Completed,2022-04-01,09:30,09:30,MAG,09:38,00:00,09:55,P,Kings,11203,Kings,11226,0.900000,Friday,Weekday
5,38,38611,S,Primary,Completed,2022-04-01,11:15,11:15,MVT,11:20,11:15,12:46,P,Kings,11203,New York,10004,8.310000,Friday,Weekday
6,39,38611,S,Primary,Completed,2022-04-01,21:10,21:10,MVT,21:42,21:40,23:03,P,New York,10004,Kings,11203,8.310000,Friday,Weekday
7,53,35870,S,Broker,Authorized,2022-04-01,12:30,12:30,VBH,12:44,12:14,13:57,P,New York,10004,Queens,11426,16.370001,Friday,Weekday
8,56,4798,S,Primary,Completed,2022-04-01,07:15,07:15,MVT,07:20,07:14,07:32,P,Kings,11221,Kings,11213,1.130000,Friday,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164000,324894,23261,S,E-Hail,Authorized,2022-04-08,04:05,04:05,LIS,00:00,00:00,00:00,P,Kings,11214,Kings,11230,3.220000,Friday,Weekday
1164001,196088,32341,S,Broker,Authorized,2022-04-04,20:10,20:10,VBG,00:00,20:30,20:52,P,New York,10032,Bronx,10461,5.490000,Monday,Weekday
1164002,670483,28744,S,E-Hail,Authorized,2022-04-17,09:59,09:59,LIS,00:00,00:00,00:00,P,Kings,11233,Queens,11419,6.230000,Sunday,Weekend
1164005,155313,34239,S,E-Hail,Authorized,2022-04-04,12:15,12:15,LIS,00:00,00:00,00:00,P,New York,10065,Queens,11434,16.570000,Monday,Weekday


## Intraborough Trips

- The mean trip distance, and total number of trips, for trips starting or ending in the respective ZIP
code, that ALSO ended or started within the same borough as that ZIP code (i.e., so called “intraborough” trips)

### 11201 

#### Starting Zip Code

In [46]:
brooklyn_zip = df4.loc[df4['PickZip'] == 11201]

In [47]:
brooklyn_zip2 = brooklyn_zip.loc[brooklyn_zip["DropCounty"].isin(['Kings'])]

In [48]:
brooklyn_start_zip = brooklyn_zip2.groupby(["PickZip", "DropCounty"]).agg({"TripMiles": "mean", 
                                                                           "PickZip": "count"}).round(2)

In [49]:
brooklyn_start_zip

,,TripMiles,PickZip
PickZip,DropCounty,,
11201,Kings,4.78,5015


#### Ending Zip Code

In [50]:
brooklyn_zip3 = df4.loc[df4['DropZip'] == 11201]

In [51]:
brooklyn_zip4 = brooklyn_zip3.loc[brooklyn_zip3["PickCounty"].isin(['Kings'])]

In [52]:
brooklyn_end_zip = brooklyn_zip4.groupby(["DropZip", "PickCounty"]).agg({"TripMiles": "mean", 
                                                                         "DropZip": "count"}).round(2)

In [53]:
brooklyn_end_zip

,,TripMiles,DropZip
DropZip,PickCounty,,
11201,Kings,4.77,5678


### 10306

#### Starting Zip Code

In [54]:
richmond_zip = df4.loc[df4['PickZip'] == 10306]

In [55]:
richmond_zip2 = richmond_zip.loc[richmond_zip["DropCounty"].isin(['Richmond'])]

In [56]:
richmond_start_zip = richmond_zip2.groupby(["PickZip", "DropCounty"]).agg({"TripMiles": "mean", 
                                                                           "PickZip": "count"}).round(2)

In [57]:
richmond_start_zip

,,TripMiles,PickZip
PickZip,DropCounty,,
10306,Richmond,3.87,2205


#### Ending Zip Code

In [58]:
richmond_zip3 = df4.loc[df4['DropZip'] == 10306]

In [59]:
richmond_zip4 = richmond_zip3.loc[richmond_zip3["PickCounty"].isin(['Richmond'])]

In [60]:
richmond_end_zip = richmond_zip4.groupby(["DropZip", "PickCounty"]).agg({"TripMiles": "mean", 
                                                                         "DropZip": "count"}).round(2)

In [61]:
richmond_end_zip

,,TripMiles,DropZip
DropZip,PickCounty,,
10306,Richmond,3.88,2145


### 10469

#### Starting Zip Code

In [62]:
bronx_zip = df4.loc[df4['PickZip'] == 10469]

In [63]:
bronx_zip2 = bronx_zip.loc[bronx_zip["DropCounty"].isin(['Bronx'])]

In [64]:
bronx_start_zip = bronx_zip2.groupby(["PickZip", "DropCounty"]).agg({"TripMiles": "mean", 
                                                                     "PickZip": "count"}).round(2)

In [65]:
bronx_start_zip

,,TripMiles,PickZip
PickZip,DropCounty,,
10469,Bronx,3.11,2234


#### Ending Zip Code

In [66]:
bronx_zip3 = df4.loc[df4['DropZip'] == 10469]

In [67]:
bronx_zip4 = bronx_zip3.loc[bronx_zip3["PickCounty"].isin(['Bronx'])]

In [68]:
bronx_end_zip = bronx_zip4.groupby(["DropZip", "PickCounty"]).agg({"TripMiles": "mean", 
                                                                   "DropZip": "count"}).round(2)

In [69]:
bronx_end_zip

,,TripMiles,DropZip
DropZip,PickCounty,,
10469,Bronx,3.1,2220


### 11694

#### Starting Zip Code

In [70]:
queens_zip = df4.loc[df4['PickZip'] == 11694]

In [71]:
queens_zip2 = queens_zip.loc[queens_zip["DropCounty"].isin(['Queens'])]

In [72]:
queens_start_zip = queens_zip2.groupby(["PickZip", "DropCounty"]).agg({"TripMiles": "mean", 
                                                                       "PickZip": "count"}).round(2)

In [73]:
queens_start_zip

,,TripMiles,PickZip
PickZip,DropCounty,,
11694,Queens,8.24,449


#### Ending Zip Code

In [74]:
queens_zip3 = df4.loc[df4['DropZip'] == 11694]

In [75]:
queens_zip4 = queens_zip3.loc[queens_zip3["PickCounty"].isin(['Queens'])]

In [76]:
queens_end_zip = queens_zip4.groupby(["DropZip", "PickCounty"]).agg({"TripMiles": "mean", 
                                                                     "DropZip": "count"}).round(2)

In [77]:
queens_end_zip

,,TripMiles,DropZip
DropZip,PickCounty,,
11694,Queens,8.26,440


## Table Summary

In [78]:
intraborough_starting_zip_table = pd.concat([brooklyn_start_zip, 
                                             bronx_start_zip, 
                                             richmond_start_zip, 
                                             queens_start_zip])

In [79]:
intraborough_starting_zip_table.rename(columns= {'TripMiles': 'Mean Trip Distance', 
                                                 'PickZip': 'Total Number of Successful Trips'})

,,Mean Trip Distance,Total Number of Successful Trips
PickZip,DropCounty,,
11201,Kings,4.78,5015
10469,Bronx,3.11,2234
10306,Richmond,3.87,2205
11694,Queens,8.24,449


In [80]:
intraborough_ending_zip_table = pd.concat([brooklyn_end_zip, 
                                             bronx_end_zip, 
                                             richmond_end_zip, 
                                             queens_end_zip])

In [81]:
intraborough_ending_zip_table.rename(columns= {'TripMiles': 'Mean Trip Distance', 
                                                 'DropZip': 'Total Number of Successful Trips'})

,,Mean Trip Distance,Total Number of Successful Trips
DropZip,PickCounty,,
11201,Kings,4.77,5678
10469,Bronx,3.10,2220
10306,Richmond,3.88,2145
11694,Queens,8.26,440


## Interborough Trips

- The mean trip distance, and total number of trips, for trips starting or ending in the respective ZIP
code, that ended or started or ended in Manhattan (i.e., “inter-borough” trips)

In [82]:
df4

,TripId,Clientid,Status,ProviderType,Outcome,Tripdate,RequestTime,PromiseTime,ProviderId,APtime,APtime1,ADtime,Anchor,PickCounty,PickZip,DropCounty,DropZip,TripMiles,day_of_week,Weekday/Weekend
4,21,36285,S,Primary,Completed,2022-04-01,09:30,09:30,MAG,09:38,00:00,09:55,P,Kings,11203,Kings,11226,0.900000,Friday,Weekday
5,38,38611,S,Primary,Completed,2022-04-01,11:15,11:15,MVT,11:20,11:15,12:46,P,Kings,11203,New York,10004,8.310000,Friday,Weekday
6,39,38611,S,Primary,Completed,2022-04-01,21:10,21:10,MVT,21:42,21:40,23:03,P,New York,10004,Kings,11203,8.310000,Friday,Weekday
7,53,35870,S,Broker,Authorized,2022-04-01,12:30,12:30,VBH,12:44,12:14,13:57,P,New York,10004,Queens,11426,16.370001,Friday,Weekday
8,56,4798,S,Primary,Completed,2022-04-01,07:15,07:15,MVT,07:20,07:14,07:32,P,Kings,11221,Kings,11213,1.130000,Friday,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164000,324894,23261,S,E-Hail,Authorized,2022-04-08,04:05,04:05,LIS,00:00,00:00,00:00,P,Kings,11214,Kings,11230,3.220000,Friday,Weekday
1164001,196088,32341,S,Broker,Authorized,2022-04-04,20:10,20:10,VBG,00:00,20:30,20:52,P,New York,10032,Bronx,10461,5.490000,Monday,Weekday
1164002,670483,28744,S,E-Hail,Authorized,2022-04-17,09:59,09:59,LIS,00:00,00:00,00:00,P,Kings,11233,Queens,11419,6.230000,Sunday,Weekend
1164005,155313,34239,S,E-Hail,Authorized,2022-04-04,12:15,12:15,LIS,00:00,00:00,00:00,P,New York,10065,Queens,11434,16.570000,Monday,Weekday


### 11201

#### Starting Zip Code

In [83]:
bk_man_zip = df4.loc[df4['PickZip'] == 11201]

In [84]:
bk_man_zip2 = bk_man_zip.loc[bk_man_zip["DropCounty"].isin(['New York'])]

In [85]:
bk_man_start_zip = bk_man_zip2.groupby(["PickZip", "DropCounty"]).agg({"TripMiles": "mean", 
                                                                     "PickZip": "count"}).round(2)

In [86]:
bk_man_start_zip

,,TripMiles,PickZip
PickZip,DropCounty,,
11201,New York,6.73,877


#### Ending Zip Code

In [87]:
bk_man_zip3 = df4.loc[df4['DropZip'] == 11201]

In [88]:
bk_man_zip4 = bk_man_zip3.loc[bk_man_zip3["PickCounty"].isin(['New York'])]

In [89]:
bk_man_end_zip = bk_man_zip4.groupby(["DropZip", "PickCounty"]).agg({"TripMiles": "mean", 
                                                                     "DropZip": "count"}).round(2)

In [90]:
bk_man_end_zip

,,TripMiles,DropZip
DropZip,PickCounty,,
11201,New York,6.86,920


### 10306

#### Starting Zip Code

In [91]:
rich_man_zip = df4.loc[df4['PickZip'] == 10306]

In [92]:
rich_man_zip2 = rich_man_zip.loc[rich_man_zip["DropCounty"].isin(['New York'])]

In [93]:
rich_man_start_zip = rich_man_zip2.groupby(["PickZip", "DropCounty"]).agg({"TripMiles": "mean", 
                                                                           "PickZip": "count"}).round(2)

In [94]:
richmond_start_zip

,,TripMiles,PickZip
PickZip,DropCounty,,
10306,Richmond,3.87,2205


#### Ending Zip Code

In [95]:
rich_man_zip3 = df4.loc[df4['DropZip'] == 10306]

In [96]:
rich_man_zip4 = rich_man_zip3.loc[rich_man_zip3["PickCounty"].isin(['New York'])]

In [97]:
rich_man_end_zip = rich_man_zip4.groupby(["DropZip", "PickCounty"]).agg({"TripMiles": "mean", 
                                                                     "DropZip": "count"}).round(2)

In [98]:
rich_man_end_zip

,,TripMiles,DropZip
DropZip,PickCounty,,
10306,New York,20.86,312


### 10469

#### Starting Zip Code

In [99]:
bx_man_zip = df4.loc[df4['PickZip'] == 10469]

In [100]:
bx_man_zip2 = bx_man_zip.loc[bx_man_zip["DropCounty"].isin(['New York'])]

In [101]:
bx_man_start_zip = bx_man_zip2.groupby(["PickZip", "DropCounty"]).agg({"TripMiles": "mean", 
                                                                           "PickZip": "count"}).round(2)

In [102]:
bx_man_start_zip

,,TripMiles,PickZip
PickZip,DropCounty,,
10469,New York,10.97,1065


#### Ending Zip Code

In [103]:
bx_man_zip3 = df4.loc[df4['DropZip'] == 10469]

In [104]:
bx_man_zip4 = bx_man_zip3.loc[bx_man_zip3["PickCounty"].isin(['New York'])]

In [105]:
bx_man_end_zip = bx_man_zip4.groupby(["DropZip", "PickCounty"]).agg({"TripMiles": "mean", 
                                                                     "DropZip": "count"}).round(2)

In [106]:
bx_man_end_zip

,,TripMiles,DropZip
DropZip,PickCounty,,
10469,New York,10.88,1018


### 11694

#### Starting Zip Code

In [107]:
qns_man_zip = df4.loc[df4['PickZip'] == 11694]

In [108]:
qns_man_zip2 = qns_man_zip.loc[qns_man_zip["DropCounty"].isin(['New York'])]

In [109]:
qns_man_start_zip = qns_man_zip2.groupby(["PickZip", "DropCounty"]).agg({"TripMiles": "mean", 
                                                                           "PickZip": "count"}).round(2)

In [110]:
qns_man_start_zip

,,TripMiles,PickZip
PickZip,DropCounty,,
11694,New York,19.76,143


#### Ending Zip Code

In [111]:
qns_man_zip3 = df4.loc[df4['DropZip'] == 11694]

In [112]:
qns_man_zip4 = qns_man_zip3.loc[qns_man_zip3["PickCounty"].isin(['New York'])]

In [113]:
qns_man_end_zip = qns_man_zip4.groupby(["DropZip", "PickCounty"]).agg({"TripMiles": "mean", 
                                                                     "DropZip": "count"}).round(2)

In [114]:
qns_man_end_zip

,,TripMiles,DropZip
DropZip,PickCounty,,
11694,New York,19.96,133


## Table Summary

In [115]:
interborough_starting_zip_table = pd.concat([bk_man_start_zip, 
                                             bx_man_start_zip, 
                                             rich_man_start_zip, 
                                             qns_man_start_zip])

In [116]:
interborough_starting_zip_table.rename(columns= {'TripMiles': 'Mean Trip Distance', 
                                                 'PickZip': 'Total Number of Successful Trips'})

,,Mean Trip Distance,Total Number of Successful Trips
PickZip,DropCounty,,
11201,New York,6.73,877
10469,New York,10.97,1065
10306,New York,20.84,355
11694,New York,19.76,143


In [117]:
interborough_ending_zip_table = pd.concat([bk_man_end_zip, 
                                             bx_man_end_zip, 
                                             rich_man_end_zip, 
                                             qns_man_end_zip])

In [118]:
interborough_ending_zip_table.rename(columns= {'TripMiles': 'Mean Trip Distance', 
                                                 'DropZip': 'Total Number of Successful Trips'})

,,Mean Trip Distance,Total Number of Successful Trips
DropZip,PickCounty,,
11201,New York,6.86,920
10469,New York,10.88,1018
10306,New York,20.86,312
11694,New York,19.96,133


## Total Starting or Ending in Same Zipcode

- The overall mean trip distance and total number of trips for trips starting or ending in the
respective ZIP code.

### 11201

#### Starting Zip Code

In [119]:
total_pick_zip1 = df4.loc[df4['PickZip'] == 11201]

In [120]:
total_start_zip11201 = total_pick_zip1.groupby(["PickZip"]).agg({"TripMiles": "mean", 
                                                                           "PickZip": "count"}).round(2)

In [121]:
total_start_zip11201

,TripMiles,PickZip
PickZip,,
11201,7.03,7579


#### Ending Zip Code

In [122]:
total_drop_zip2 = df4.loc[df4['DropZip'] == 11201]

In [123]:
total_end_zip11201 = total_drop_zip2.groupby(["DropZip"]).agg({"TripMiles": "mean", 
                                                                           "DropZip": "count"}).round(2)

In [124]:
total_end_zip11201

,TripMiles,DropZip
DropZip,,
11201,6.89,8375


### 10306

#### Starting Zip Code

In [125]:
total_pick_zip3 = df4.loc[df4['PickZip'] == 10306]

In [126]:
total_start_zip10306 = total_pick_zip3.groupby(["PickZip"]).agg({"TripMiles": "mean", 
                                                                           "PickZip": "count"}).round(2)

In [127]:
total_start_zip10306

,TripMiles,PickZip
PickZip,,
10306,7.99,3181


#### Ending Zip Code

In [128]:
total_drop_zip4 = df4.loc[df4['DropZip'] == 10306]

In [129]:
total_end_zip10306 = total_drop_zip4.groupby(["DropZip"]).agg({"TripMiles": "mean", 
                                                                           "DropZip": "count"}).round(2)

In [130]:
total_end_zip10306

,TripMiles,DropZip
DropZip,,
10306,7.74,3040


### 10469

#### Starting Zip Code

In [131]:
total_pick_zip5 = df4.loc[df4['PickZip'] == 10469]

In [132]:
total_start_zip10469 = total_pick_zip5.groupby(["PickZip"]).agg({"TripMiles": "mean", 
                                                                           "PickZip": "count"}).round(2)

In [133]:
total_start_zip10469

,TripMiles,PickZip
PickZip,,
10469,7.3,3754


#### Ending Zip Code

In [134]:
total_drop_zip6 = df4.loc[df4['DropZip'] == 10469]

In [135]:
total_end_zip10469 = total_drop_zip6.groupby(["DropZip"]).agg({"TripMiles": "mean", 
                                                                           "DropZip": "count"}).round(2)

In [136]:
total_end_zip10469

,TripMiles,DropZip
DropZip,,
10469,7.24,3693


### 11694

#### Starting Zip Code

In [137]:
total_pick_zip7 = df4.loc[df4['PickZip'] == 11694]

In [138]:
total_start_zip11694 = total_pick_zip7.groupby(["PickZip"]).agg({"TripMiles": "mean", 
                                                                           "PickZip": "count"}).round(2)

In [139]:
total_start_zip11694

,TripMiles,PickZip
PickZip,,
11694,11.22,876


#### Ending Zip Code

In [140]:
total_drop_zip8 = df4.loc[df4['DropZip'] == 11694]

In [141]:
total_end_zip11694 = total_drop_zip8.groupby(["DropZip"]).agg({"TripMiles": "mean", 
                                                                           "DropZip": "count"}).round(2)

In [142]:
total_end_zip11694

,TripMiles,DropZip
DropZip,,
11694,11.1,854


## Table Summary

In [143]:
starting_zip_table = pd.concat([total_start_zip11201, total_start_zip10306, 
                                total_start_zip10469, total_start_zip11694])

In [144]:
starting_zip_table.rename(columns= {'TripMiles': 'Mean Trip Distance', 
                                                 'PickZip': 'Total Number of Successful Trips'})

,Mean Trip Distance,Total Number of Successful Trips
PickZip,,
11201,7.03,7579
10306,7.99,3181
10469,7.30,3754
11694,11.22,876


In [145]:
ending_zip_table = pd.concat([total_end_zip11201, total_end_zip10306, 
                              total_end_zip10469, total_end_zip11694])

In [146]:
ending_zip_table.rename(columns= {'TripMiles': 'Mean Trip Distance', 
                                                 'DropZip': 'Total Number of Successful Trips'})

,Mean Trip Distance,Total Number of Successful Trips
DropZip,,
11201,6.89,8375
10306,7.74,3040
10469,7.24,3693
11694,11.10,854


# Findings

In a few sentences -- comments in the code are OK -- given the results from these 4 ZIP codes, what can
you say about how costly it is to serve Paratransit customers based on their trip geography? 

Travel from Queens, specifically from the ZIP code 11694, appears to be more costly than the other ZIP codes. When analyzing Intraborough and overall successful trips starting or ending in each respective ZIP code, 11694 had the highest mean distance but the lowest number of successful trips. A similar pattern arises when examining Interborough trips starting or ending in Manhattan, with 10306 and 11694 having the highest mean distance and lowest number of successful trips compared to the other two respective ZIP codes. ZIP code 11201 had the highest total of successful Intraborough and overall starting or ending trips, while having the second-highest total of successful Interborough trips. In terms of mean trip distance, 11201 had the lowest amount of average trip distance that started or ended in the respective ZIP code, as well as having the lowest average trip distance for Interborough trips starting or ending in 11201, and the second-lowest average for Intraborough trips that started or ended in the ZIP code. It appears that traveling from or to ZIP code 11201 is the least costly and traveling from or to ZIP code 11694 is the most costly. However, this assessment may be the result of the disparity in the total number of successful trips in each ZIP code. 